# 飞书机器人上手教程 (2) — 接收消息与路由

本 Notebook 演示如何让机器人**接收**用户消息，并通过装饰器进行路由分发。

## 接收消息的两种模式

| 模式 | 特点 | 适合场景 |
|------|------|---------|
| **WebSocket 长连接** | 无需公网 URL，SDK 主动连接飞书 | 本地开发、内网部署 |
| **Webhook (Flask/FastAPI)** | 需要 HTTPS 公网地址 | 生产环境 |

**本教程使用 WebSocket 模式**，适合在本地直接运行。

## 前置配置

在飞书开放平台「事件订阅」中选择：**使用长连接接收事件**  
然后订阅 `im.message.receive_v1` 事件（接收消息）。

In [1]:
from chattool.tools.lark import LarkBot

TEST_USER = "rexwzh"
ID_TYPE   = "user_id"

## 1. 最简单的消息处理器

In [3]:
bot = LarkBot()

@bot.on_message
def echo(ctx):
    """把用户发的内容原样回复"""
    ctx.reply(f"你说了：{ctx.text}")

# 在 Notebook 中用后台线程启动，不会阻塞后续单元格
t = bot.start_background()
print("✅ 机器人已在后台启动，向机器人发消息试试吧！")
print("（若要停止，重启 Kernel 即可）")

✅ 机器人已在后台启动，向机器人发消息试试吧！
（若要停止，重启 Kernel 即可）


## 2. MessageContext — 消息上下文对象

处理函数收到的 `ctx` 包含所有消息信息：

In [4]:
# 演示 ctx 的所有属性（不启动机器人，直接构造一个假 ctx 查看）
import json
from unittest.mock import MagicMock
from chattool.tools.lark.context import MessageContext

def make_fake_ctx(text="你好", chat_type="p2p", open_id="rexwzh"):
    msg = MagicMock()
    msg.message_type = "text"
    msg.content = json.dumps({"text": text})
    msg.message_id = "om_demo123"
    msg.chat_id = "oc_demo456"
    msg.chat_type = chat_type
    msg.thread_id = None

    sid = MagicMock(); sid.open_id = open_id
    sender = MagicMock(); sender.sender_id = sid; sender.sender_type = "user"
    event = MagicMock(); event.message = msg; event.sender = sender
    data = MagicMock(); data.event = event

    mock_bot = MagicMock()
    return MessageContext(mock_bot, data)

ctx = make_fake_ctx(text="你好，机器人！", chat_type="group", open_id="rexwzh")

print(f"ctx.text       = {ctx.text!r}")
print(f"ctx.msg_type   = {ctx.msg_type!r}")
print(f"ctx.sender_id  = {ctx.sender_id!r}")
print(f"ctx.chat_id    = {ctx.chat_id!r}")
print(f"ctx.chat_type  = {ctx.chat_type!r}")
print(f"ctx.is_group   = {ctx.is_group}")
print(f"ctx.message_id = {ctx.message_id!r}")
print(f"ctx.thread_id  = {ctx.thread_id!r}")

ctx.text       = '你好，机器人！'
ctx.msg_type   = 'text'
ctx.sender_id  = 'rexwzh'
ctx.chat_id    = 'oc_demo456'
ctx.chat_type  = 'group'
ctx.is_group   = True
ctx.message_id = 'om_demo123'
ctx.thread_id  = None


## 3. 指令路由（@bot.command）

`/help`、`/status` 等以斜杠开头的指令会优先匹配，不会触发 `on_message` 兜底处理器。

In [5]:
bot2 = LarkBot()

@bot2.command("/help")
def on_help(ctx):
    ctx.reply(
        "📖 可用指令：\n"
        "  /help   — 显示帮助\n"
        "  /status — 查看服务状态\n"
        "  /clear  — 清除对话记忆"
    )

@bot2.command("/status")
def on_status(ctx):
    import time
    ctx.reply(f"✅ 服务运行正常\n时间：{time.strftime('%Y-%m-%d %H:%M:%S')}")

@bot2.on_message
def on_other(ctx):
    ctx.reply(f"收到你的消息：{ctx.text}\n（发送 /help 查看可用指令）")

# 模拟派发，验证路由正确
def simulate(b, text, chat_type="p2p"):
    msg = MagicMock()
    msg.message_type = "text"
    msg.content = json.dumps({"text": text})
    msg.message_id = "om_sim"; msg.chat_id = "p2p_sim"
    msg.chat_type = chat_type; msg.thread_id = None
    sid = MagicMock(); sid.open_id = "rexwzh"
    sender = MagicMock(); sender.sender_id = sid; sender.sender_type = "user"
    event = MagicMock(); event.message = msg; event.sender = sender
    data = MagicMock(); data.event = event
    b._dispatch_message(data)

# 测试 /help 指令
simulate(bot2, "/help")
print("✅ /help 调用了 on_help（check: bot2 的 reply 被调用）")

# 测试普通消息
simulate(bot2, "随便说点什么")
print("✅ 普通消息走了 on_other 兜底")

[Lark] [2026-02-26 04:11:45,853] [ERROR] client.im.v1.message.reply failed, code: 99992354, msg: The request you send is not a valid {open_message_id} or not exists, and the example value is {om_da5****************dfe}. Invalid ids: [om_sim] . Please see field_violations for details and refer to API documentation to correct accordingly and resubmit, log_id: 202602260411455A26040A553752FDE10C
✅ /help 调用了 on_help（check: bot2 的 reply 被调用）
[Lark] [2026-02-26 04:11:46,048] [ERROR] client.im.v1.message.reply failed, code: 99992354, msg: The request you send is not a valid {open_message_id} or not exists, and the example value is {om_da5****************dfe}. Invalid ids: [om_sim] . Please see field_violations for details and refer to API documentation to correct accordingly and resubmit, log_id: 20260226041145BCE369D4113A89F7BD8A
✅ 普通消息走了 on_other 兜底


## 4. 正则匹配（@bot.regex）

In [6]:
bot3 = LarkBot()

@bot3.regex(r"^查询\s+(.+)$")
def on_query(ctx):
    keyword = ctx._match.group(1)  # 正则捕获组
    ctx.reply(f"🔍 正在查询：**{keyword}**...")

@bot3.regex(r"^(天气|温度)\s*(\S+)?")
def on_weather(ctx):
    city = ctx._match.group(2) or "北京"
    ctx.reply(f"🌤 {city}今日天气：晴，22°C")

@bot3.on_message
def fallback(ctx):
    ctx.reply("不明白你说的，试试：查询 <关键词> 或 天气 <城市>")

# 验证
for text in ["查询 Python asyncio", "天气 上海", "你好"]:
    print(f"输入: {text!r}")
    simulate(bot3, text)
print("\n✅ 正则路由工作正常（实际回复会发到飞书）")

输入: '查询 Python asyncio'
[Lark] [2026-02-26 04:11:52,761] [ERROR] client.im.v1.message.reply failed, code: 99992354, msg: The request you send is not a valid {open_message_id} or not exists, and the example value is {om_da5****************dfe}. Invalid ids: [om_sim] . Please see field_violations for details and refer to API documentation to correct accordingly and resubmit, log_id: 2026022604115256FB2CA0CDE688E74A35
输入: '天气 上海'
[Lark] [2026-02-26 04:11:52,991] [ERROR] client.im.v1.message.reply failed, code: 99992354, msg: The request you send is not a valid {open_message_id} or not exists, and the example value is {om_da5****************dfe}. Invalid ids: [om_sim] . Please see field_violations for details and refer to API documentation to correct accordingly and resubmit, log_id: 202602260411520347BE2BD4A48CE53215
输入: '你好'
[Lark] [2026-02-26 04:11:53,225] [ERROR] client.im.v1.message.reply failed, code: 99992354, msg: The request you send is not a valid {open_message_id} or not exists,

## 5. 群聊 / 私聊过滤

In [7]:
bot4 = LarkBot()

@bot4.on_message(group_only=True)
def group_handler(ctx):
    # 只处理群消息
    ctx.reply(f"[群消息] 收到 @{ctx.sender_id} 的消息")

@bot4.on_message(private_only=True)
def private_handler(ctx):
    # 只处理私聊
    ctx.reply(f"[私聊] 你好，有什么可以帮到你？")

# 群消息
simulate(bot4, "大家好", chat_type="group")
print("群消息 → group_handler")

# 私聊消息
simulate(bot4, "你好", chat_type="p2p")
print("私聊消息 → private_handler")

[Lark] [2026-02-26 04:12:00,001] [ERROR] client.im.v1.message.reply failed, code: 99992354, msg: The request you send is not a valid {open_message_id} or not exists, and the example value is {om_da5****************dfe}. Invalid ids: [om_sim] . Please see field_violations for details and refer to API documentation to correct accordingly and resubmit, log_id: 20260226041159CF548DBBBEDB57E0EE64
群消息 → group_handler
[Lark] [2026-02-26 04:12:00,227] [ERROR] client.im.v1.message.reply failed, code: 99992354, msg: The request you send is not a valid {open_message_id} or not exists, and the example value is {om_da5****************dfe}. Invalid ids: [om_sim] . Please see field_violations for details and refer to API documentation to correct accordingly and resubmit, log_id: 20260226041200D3CB9E36366B412E14D7
私聊消息 → private_handler


## 6. 机器人进群事件

In [8]:
bot5 = LarkBot()

@bot5.on_bot_added
def welcome(chat_id):
    """机器人被拉入群时自动发欢迎消息"""
    bot5.send_text(chat_id, "chat_id",
        "大家好！我是 AI 助手 🤖\n"
        "发送 /help 查看可用指令。"
    )

print("✅ on_bot_added 已注册，当机器人进群时将自动发送欢迎消息")

✅ on_bot_added 已注册，当机器人进群时将自动发送欢迎消息


## 7. 启动机器人（真实运行）

⚠️ 执行下面单元格后机器人开始监听消息，向机器人发任意消息测试效果。

> 重启 Kernel 即可停止。

In [ ]:
from chattool.tools.lark import LarkBot

bot_live = LarkBot()

@bot_live.command("/help")
def help_cmd(ctx):
    ctx.reply("📖 指令列表：/help  /status")

@bot_live.command("/status")
def status_cmd(ctx):
    import time
    ctx.reply(f"✅ 在线 | {time.strftime('%H:%M:%S')}")

@bot_live.on_message
def echo(ctx):
    ctx.reply(f"📨 收到：{ctx.text}")

# 后台启动（不阻塞 Notebook）
bot_live.start_background()
print("🚀 机器人已启动，发消息试试吧！")

## 小结

| 装饰器 | 匹配方式 |
|--------|----------|
| `@bot.on_message` | 兜底，匹配所有消息 |
| `@bot.on_message(group_only=True)` | 仅群消息 |
| `@bot.on_message(private_only=True)` | 仅私聊 |
| `@bot.command("/cmd")` | 精确匹配斜杠指令（优先级最高）|
| `@bot.regex(r"pattern")` | 正则匹配消息文本 |
| `@bot.on_bot_added` | 机器人进群事件 |

下一篇：[03 — AI 对话集成](lark_03_ai_chat.ipynb)